In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, SGDRegressor, HuberRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, ShuffleSplit
import sklearn.metrics as mr
from scipy.sparse import hstack
from string import punctuation

def make_words(s: str) -> str:
    s = "".join(c for c in s if c not in punctuation)
    return "".join(s.strip('-;:!,."nan♪¶†—-'))

In [2]:
df = pd.read_csv('train.tsv', 
                   names=['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence'],
                   header=None, sep='\t', quoting=3)

In [3]:
data_final = pd.read_csv('final.tsv',
                         sep='\t', quoting=3, error_bad_lines=False, header=None,
                         names=['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply'])

In [4]:
data_final['context_2'] = data_final['context_2'].astype('str').apply(make_words)
data_final['context_1'] = data_final['context_1'].astype('str').apply(make_words)
data_final['context_0'] = data_final['context_0'].astype('str').apply(make_words)
data_final['reply'] = data_final['reply'].astype('str').apply(make_words)

context_2 = data_final['context_2']
context_1 = data_final['context_1']
context_0 = data_final['context_0']
context_total = context_2 + context_1 + context_0
data_final.insert(loc=4, column='context_total', value=context_total)

data_final = data_final.drop(['context_2', 'context_1', 'context_0'], axis=1)

data_final['context_total'] = data_final['context_total'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
# data_final['context_2'] = data_final['context_2'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
# data_final['context_1'] = data_final['context_1'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
# data_final['context_0'] = data_final['context_0'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
data_final['reply'] = data_final['reply'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')

reply_id = data_final['reply_id']
data_final = data_final.drop(['reply_id'], axis=1)
data_final.insert(loc=1, column='reply_id', value=reply_id)

In [5]:
data_final.sample(10)

,context_id,reply_id,context_total,reply
21441,56259420367731,1,пожалуйста джонас называйте меня дениз я пре...,буду знать
33966,90723199992080,4,чтобы убедиться что ваши людиявляются лучшим ...,нет президент не думаю что вы сможете
76930,205369554706218,2,никакие,ты помер
41570,111466072549048,4,ты подвергал детей опасности уходи немедленно ...,вечером
8205,22021000338340,3,хорошо и так что мы будем делать дальше знае...,в течение последних нескольких месяцев я вел с...
6568,17631231360111,2,ни в чём не обвиняю я говорю что ты умный,очень умен умна
89450,239182021063826,5,сначала они берутся за ножи потом за пистолет...,не мне и тебе
76413,203757812151067,2,запускайте подземный бур начинайте вход в земн...,мы проверяем пятый этаж
77565,206979100151639,2,правильно тянуть соломинку с профессиональным ...,потому что это вздор
40213,107816846678577,5,если она узнает что меня допрашивали о парне ...,я спокоен


In [6]:
df['context_2'] = df['context_2'].astype('str').apply(make_words)
df['context_1'] = df['context_1'].astype('str').apply(make_words)
df['context_0'] = df['context_0'].astype('str').apply(make_words)
df['reply'] = df['reply'].astype('str').apply(make_words)

context_2 = df['context_2']
context_1 = df['context_1']
context_0 = df['context_0']
context_total = context_2 + context_1 + context_0
df.insert(loc=4, column='context_total', value=context_total)

df = df.drop(['context_2', 'context_1', 'context_0'], axis=1)

df['context_total'] = df['context_total'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
# df['context_2'] = df['context_2'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
# df['context_1'] = df['context_1'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
# df['context_0'] = df['context_0'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')
df['reply'] = df['reply'].str.strip().str.replace('€', 'я').str.replace('ќх', 'ох').str.replace('ќт', 'от').str.replace('ќн', 'он').str.replace('ѕою', 'бою').str.replace('ѕег', 'бег').str.replace('ѕросай', 'бросай').str.replace('ѕоб', 'боб').str.replace('ѕольше', 'больше').str.replace('sа', 'па').str.replace('ѕо', 'по').str.replace('ѕр', 'пр').str.replace('ќ', 'н').str.replace('¬', 'в').str.replace('" то', 'что').str.replace('" ы', 'ты').str.replace("' о", 'хо').str.replace('" а', 'та').str.replace('ƒ', 'д').str.replace('√', 'г').str.replace('ьi', 'ы').str.replace('ћад', 'лад').str.replace('ћ', 'м').str.replace('hо', 'но').str.replace('hу', 'ну').str.replace('∆', 'ж').str.replace('j', 'a').str.replace('≈', 'е').str.replace('tр', 'тр').str.replace('bс', 'вс').str.replace('bи', 'ви')

reply_id = df['reply_id']
df = df.drop(['reply_id'], axis=1)
df.insert(loc=1, column='reply_id', value=reply_id)

In [7]:
# df['label'] = df['label'].replace({'good': 1, 'neutral': 0.1, 'bad': 0.01})
# df['label'] = df['label'].replace({'good': 2, 'neutral': 1, 'bad': 0})
# df['label'] = df['label'].replace({'good': 2, 'neutral': 1, 'bad': -1})
# df['label'] = df['label'].replace({'good': 2, 'neutral': 1, 'bad': 0.5})
# df['label'] = df['label'].replace({'good': 1, 'neutral': 0.5, 'bad': 0})
df['label'] = df['label'].replace({'good': 1, 'neutral': 0.5, 'bad': 0.1})


# target = ((df['label'] * df['confidence']) + df['label']) / 2
# target = df['label'] * df['confidence']
# target = (df['label'] * df['confidence']) * (df['label'] + df['confidence']) # 84652
target = ((df['confidence'] + 1) ** df['label']) - 1 # 84734
df.insert(loc=len(df.columns), column='target', value=target)

In [8]:
df.sample(10)

,context_id,reply_id,context_total,reply,label,confidence,target
8594,24053383406932,0,без продовольствия они умрут через несколько дней,эти люди должны умереть,0.1,0.979127,0.070650
37566,108286787571541,0,возможно у вас это уже давно цирроз может быт...,нет ты получишь его в другом месте типа желу...,0.1,0.883504,0.065361
66977,192805717488846,1,чего ты уставилась куда ты идёшь привет,приветик,1.0,0.951586,0.951586
27066,78279840145002,1,это конечно очень печально но по моему это...,к полиции у всех особое отношение да,0.1,0.946034,0.068846
31147,90261473486218,2,крутящий момент,ну инструмент да вынули,0.1,0.945948,0.068841
32205,93011226033624,3,тебе просто надо быть сильнее этоготебе легко ...,я могу рассказать тебе все,1.0,0.963285,0.963285
77950,225045907767189,0,купил его,купил,1.0,0.908537,0.908537
6812,18760028901671,3,сюзан что черт возьми ты делаешь когда ты с...,какие обвинения,0.1,0.559299,0.045425
78661,226709352853168,5,кармилла пайнс и ты отправил к ней хоули,да забрал,0.1,0.987342,0.071093
52701,151661888072034,4,скажи им что я сбежала я не буду лгать ради т...,ладно,0.5,0.880461,0.371299


In [120]:
# content = df['context_total'].append(df['reply']).append(data_final['context_total']).append(data_final['reply'])
# content = df['context_2'].append(df['context_1']).append(df['context_0']).append(data_final['context_2']).append(data_final['context_1']).append(data_final['context_0'])
content = df['context_total'].append(data_final['context_total'])

In [121]:
content.shape

(97533,)

In [122]:
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(1, 4)).fit(content)
# tfidf = TfidfVectorizer(ngram_range=(1, 2)).fit(content)

In [123]:
tfidf

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [1940]:
# df_cut = df[df.confidence >= 0.7]
# df_cut = df[df.confidence >= 0.9]
# df_cut = df[df.confidence >= 0.5]

In [138]:
# tfidf_2 = tfidf.transform(df_cut['reply'])
tfidf_2 = tfidf.transform(df['reply'])

tfidf_f2 = tfidf.transform(data_final['reply'])

In [139]:
X_train = hstack([tfidf_2]).tocsr()
X_final = hstack([tfidf_f2]).tocsr()

In [140]:
# y_train = df_cut['target']
y_train = df['target']

In [141]:
# train_part_size = int(0.75 * df_cut['target'].shape[0])
train_part_size = int(0.75 * df['target'].shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

In [142]:
X_train_part.shape, y_train_part.shape

((73149, 55974), (73149,))

In [143]:
from sklearn.linear_model import Ridge, SGDRegressor, HuberRegressor

# reg = Ridge(alpha=4.62, random_state=10) [model_03][85506]
# reg = SGDRegressor(max_iter=1000, power_t=0.1, random_state=10) [model_05][85334]
# reg = HuberRegressor(epsilon=1.5) [model_06][84937]

reg = Ridge(alpha=4.62, random_state=10)
reg.fit(X_train_part, y_train_part)

Ridge(alpha=4.62, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=10, solver='auto', tol=0.001)

In [144]:
reg_pred = reg.predict(X_valid)

In [145]:
print('explained_variance_score: ', mr.explained_variance_score(y_valid, reg_pred))
print('r2_score:                 ', mr.r2_score(y_valid, reg_pred))
print('mean_absolute_error:      ', mr.mean_absolute_error(y_valid, reg_pred))
print('median_absolute_error:    ', mr.median_absolute_error(y_valid, reg_pred))
print('mean_squared_error:       ', mr.mean_squared_error(y_valid, reg_pred))
print('mean_squared_log_error:   ', mr.mean_squared_log_error(y_valid, reg_pred))
print()
print('completeness_score:       ', mr.completeness_score(y_valid, reg_pred))
print('v_measure_score:          ', mr.v_measure_score(y_valid, reg_pred))
print('homogeneity_score:        ', mr.homogeneity_score(y_valid, reg_pred))
print('adjusted_rand_score:      ', mr.adjusted_rand_score(y_valid, reg_pred))

explained_variance_score:  0.003017984877768942
r2_score:                  0.003015982849911669
mean_absolute_error:       0.36029668753500876
median_absolute_error:     0.3876188045666219
mean_squared_error:        0.15012269746911475
mean_squared_log_error:    0.0715856616944283

completeness_score:        0.9649622270240542
v_measure_score:           0.8849234556501944
homogeneity_score:         0.8171453518409748
adjusted_rand_score:       0.01749696729613996


In [146]:
X_train.shape, X_final.shape

((97533, 55974), (104834, 55974))

In [147]:
reg.fit(X_train, y_train)

Ridge(alpha=4.62, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=10, solver='auto', tol=0.001)

In [148]:
y_final = reg.predict(X_final)

In [149]:
sub = pd.DataFrame()
sub['context_id'] = data_final['context_id']
sub['reply_id'] = data_final['reply_id']
sub['rank'] = y_final

In [1929]:
# ridge world n_gram 1,2
sub_1 = pd.DataFrame()
sub_1['rank'] = y_final

In [1805]:
# ridge char n_gram 1,4
sub_2 = pd.DataFrame()
sub_2['rank'] = y_final

In [2221]:
# sdg char n_gram 1,4
sub_3 = pd.DataFrame()
sub_3['rank'] = y_final

In [1953]:
# ridge 10% char n_gram 1,4 (85506)
sub_4 = pd.DataFrame()
sub_4['rank'] = y_final

In [2276]:
# sdg world n_gram 1,2
sub_5 = pd.DataFrame()
sub_5['rank'] = y_final

In [2371]:
# huber world n_gram 1,2
sub_6 = pd.DataFrame()
sub_6['rank'] = y_final

In [2392]:
# huber char n_gram 1,4
sub_7 = pd.DataFrame()
sub_7['rank'] = y_final

In [2431]:
# sub['rank'] = sub_1['rank'] + sub_2['rank'] + sub_7['rank'] + sub_6['rank']
sub['rank'] = sub_1['rank'] + sub_2['rank'] + sub_3['rank'] + sub_5['rank'] + sub_6['rank'] + sub_7['rank'] + sub_4['rank']

In [150]:
submission = sub.sort_values(by=['context_id', 'rank'], ascending=[True, False])

In [151]:
submission.head(12)

,context_id,reply_id,rank
0,4909294510,0,0.397245
1,4909294510,1,0.397245
2,4909294510,2,0.397245
3,4909294510,3,0.397245
4,4909294510,4,0.397245
5,4909294510,5,0.397245
6,15805100619,0,0.515302
7,15805100619,1,0.515302
8,15805100619,2,0.515302
9,15805100619,3,0.515302


In [57]:
del submission['rank']
submission.head(10)

,context_id,reply_id
4,4909294510,4
1,4909294510,1
2,4909294510,2
3,4909294510,3
5,4909294510,5
0,4909294510,0
11,15805100619,5
10,15805100619,4
6,15805100619,0
8,15805100619,2


In [58]:
submission.to_csv('final_151_model_11.tsv', header=None, index=False, sep=' ')